In [64]:
## Liberías

# Clásicas de análisis de datos
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

# Librerías de ML
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Métricas de modelos
from sklearn.metrics import confusion_matrix

In [65]:
# Dataset Original
df = pd.read_csv('./Dataset/Anime2.csv', index_col=0)
df.head(10)

,Expertos,Capitulos,Animes,Edad,Mujer,CantCaps,Scoremyanimelist,NEscenasCombate,NescenasEmocionantes,CalidadAnimacion,DuracionCapituloMin,RatingTV,Eleccion
1.1.1,1,1,1,42,1,575,8.0,3,2,4,41,13,0
1.1.2,1,1,2,42,1,720,6.5,4,1,7,54,27,1
1.1.3,1,1,3,42,1,148,10.0,1,0,2,30,20,0
1.1.4,1,1,4,42,1,113,4.0,2,1,8,53,15,0
1.2.1,1,2,1,42,1,575,8.0,4,1,7,59,25,0
1.2.2,1,2,2,42,1,720,6.5,4,3,6,32,19,0
1.2.3,1,2,3,42,1,148,10.0,2,3,3,44,38,1
1.2.4,1,2,4,42,1,113,4.0,4,2,4,35,37,0
1.3.1,1,3,1,42,1,575,8.0,2,1,2,45,29,0
1.3.2,1,3,2,42,1,720,6.5,2,2,4,34,32,1


In [66]:
# Cambiamos el nombre de algunas columnas para facilitar la vida

# Atributos a agrupar en una sola fila por eleccion 
cambio_nombre = {'NEscenasCombate':'Combat', 'NescenasEmocionantes':'Emotion', 'CalidadAnimacion':'Qual', 'DuracionCapituloMin':'Duration', 'RatingTV':'Rating', 'CantCaps':'Caps', 'Scoremyanimelist':'Score'}

# Cambio de nombre
df.rename(cambio_nombre, axis=1, inplace=True)

df.head()

,Expertos,Capitulos,Animes,Edad,Mujer,Caps,Score,Combat,Emotion,Qual,Duration,Rating,Eleccion
1.1.1,1,1,1,42,1,575,8.0,3,2,4,41,13,0
1.1.2,1,1,2,42,1,720,6.5,4,1,7,54,27,1
1.1.3,1,1,3,42,1,148,10.0,1,0,2,30,20,0
1.1.4,1,1,4,42,1,113,4.0,2,1,8,53,15,0
1.2.1,1,2,1,42,1,575,8.0,4,1,7,59,25,0


### Homogéneo

Formato de clases (Previo a Wide)

In [67]:
# Dropeamos la columnas que no nos sirven para el análisis homogéneo
# Expertos = indiv, capitulos = tiempo, anime = producto
df_homogeneo = df.copy() #.drop(columns=['CantCaps', 'Scoremyanimelist'])

# La elección la pasamos a categorías
df_homogeneo['Eleccion'] = df_homogeneo['Eleccion'] * df_homogeneo['Animes']

# Eliminamos anime que ya es parte de Eleccion
df_homogeneo.drop('Animes', axis=1, inplace=True)

df_homogeneo.head(12)

,Expertos,Capitulos,Edad,Mujer,Caps,Score,Combat,Emotion,Qual,Duration,Rating,Eleccion
1.1.1,1,1,42,1,575,8.0,3,2,4,41,13,0
1.1.2,1,1,42,1,720,6.5,4,1,7,54,27,2
1.1.3,1,1,42,1,148,10.0,1,0,2,30,20,0
1.1.4,1,1,42,1,113,4.0,2,1,8,53,15,0
1.2.1,1,2,42,1,575,8.0,4,1,7,59,25,0
1.2.2,1,2,42,1,720,6.5,4,3,6,32,19,0
1.2.3,1,2,42,1,148,10.0,2,3,3,44,38,3
1.2.4,1,2,42,1,113,4.0,4,2,4,35,37,0
1.3.1,1,3,42,1,575,8.0,2,1,2,45,29,0
1.3.2,1,3,42,1,720,6.5,2,2,4,34,32,2


In [68]:
# Agrupamos para juntar la información de una combinación (i,j,t) en una sola (juntar los 4 animé en una sola fila y con la elección como categórica)

# Tupla
tupla = ['Expertos', 'Capitulos'] # El animé es parte de la eleecion

# Atributos a agrupar en una sola fila por eleccion 
atributos_eleccion = ['Combat', 'Emotion', 'Qual', 'Duration', 'Rating', 'Caps', 'Score']

# Animes
animes = ['hunterX', 'dragonball', 'naruto', 'bnha']

# Grop by según tupla
gb = df_homogeneo.groupby(tupla)

# Creamos una copia donde vamos a guardar el resultado final
df_homogeneo_agrupado = df_homogeneo.copy()

# Eliminamos las filas donde no hay eleccion
df_homogeneo_agrupado = df_homogeneo_agrupado[df_homogeneo_agrupado['Eleccion'] > 0].reset_index()

# Eliminamos las columnas que agrupamos
df_homogeneo_agrupado.drop(columns=atributos_eleccion, inplace=True)

# Recorro cada anime y veo si calza con el original
for i in range(4):

    # Recorro cada atributo a renombrar
    for atr in atributos_eleccion:

        # Agrego el atributo renombrado al dataframe
        df_homogeneo_agrupado[f'{atr}.{animes[i]}'] = 0
        df_homogeneo_agrupado[f'{atr}.{animes[i]}'] = gb.nth(i).reset_index()[atr]

    # Renombramos también la columna de Eleccion
    df_homogeneo_agrupado.loc[ df_homogeneo_agrupado['Eleccion'] == i+1, 'Eleccion' ] = animes[i]

# Eliminamos la columna de indice vieja y los indices de la tupla
df_homogeneo_agrupado.drop(columns=['index', 'Expertos', 'Capitulos'], inplace=True)

# # Renombramos la columna de elección
# for i in range
# df_homogeneo_agrupado['Eleccion']

# Aliminamos las 
df_homogeneo_agrupado.head(12)

,Edad,Mujer,Eleccion,Combat.hunterX,Emotion.hunterX,Qual.hunterX,Duration.hunterX,Rating.hunterX,Caps.hunterX,Score.hunterX,...,Rating.naruto,Caps.naruto,Score.naruto,Combat.bnha,Emotion.bnha,Qual.bnha,Duration.bnha,Rating.bnha,Caps.bnha,Score.bnha
0,42,1,dragonball,3,2,4,41,13,575,8.0,...,20,148,10.0,2,1,8,53,15,113,4.0
1,42,1,naruto,4,1,7,59,25,575,8.0,...,38,148,10.0,4,2,4,35,37,113,4.0
2,42,1,dragonball,2,1,2,45,29,575,8.0,...,35,148,10.0,2,1,6,48,23,113,4.0
3,42,1,dragonball,3,1,7,39,16,575,8.0,...,23,148,10.0,2,2,8,54,6,113,4.0
4,42,1,dragonball,1,1,6,46,33,575,8.0,...,13,148,10.0,1,2,2,50,12,113,4.0
5,42,1,bnha,4,2,6,55,24,575,8.0,...,24,148,10.0,0,0,9,59,5,113,4.0
6,42,1,dragonball,4,2,6,31,22,575,8.0,...,21,148,10.0,2,0,1,45,11,113,4.0
7,42,1,dragonball,2,0,9,39,27,575,8.0,...,12,148,10.0,3,3,4,34,25,113,4.0
8,42,1,dragonball,4,2,2,51,13,575,8.0,...,34,148,10.0,2,3,5,33,35,113,4.0
9,42,1,dragonball,2,1,4,46,35,575,8.0,...,39,148,10.0,1,2,9,35,28,113,4.0


In [69]:
# Guardemos el dataset para correr los modelos en R
df_homogeneo_agrupado.to_csv('./Dataset/homogeneo.csv', index=False)